In [6]:
# %%
import pandas as pd
#import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
#import plotly.graph_objects as go
# 출력 제한 해제
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_seq_items', None)
pd.options.display.max_colwidth = 100

import warnings
warnings.filterwarnings(action='ignore') 

In [7]:
# %%
train = pd.read_csv('./data/train.csv')
test =  pd.read_csv('./data/test.csv')
trade = pd.read_csv('./data/international_trade.csv')

train.head()


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [8]:
# %%
time = pd.to_datetime(train['timestamp'].copy())

train['Date'] = pd.to_datetime(train['timestamp'])
train['week'] = train['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
train['day_name'] = train['Date'].dt.day_name()

train['year'] = train['timestamp'].apply(lambda x : int(x[0:4]))
train['month'] = train['timestamp'].apply(lambda x : int(x[5:7]))
train['day'] = train['timestamp'].apply(lambda x : int(x[8:10]))

In [9]:
# %%
import datetime
import holidays

# %%
#train = train[(train['supply(kg)'] != 0) & (train['supply(kg)'] != 0)] # 이거차이도 큼!!! 거의 rmse 1200에서 680으로 줄어듦. 근데 이건 validation에만이고 test는 이거하면 정반대로됨 ㅠ...

# %%
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
test_re = test.copy()

test_re['year'] = test_re['timestamp'].apply(lambda x : int(x[0:4]))
test_re['month'] = test_re['timestamp'].apply(lambda x : int(x[5:7]))
test_re['day'] = test_re['timestamp'].apply(lambda x : int(x[8:10]))
test_re['Date'] = pd.to_datetime(test_re['timestamp'])
test_re['week'] = test_re['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
test_re['day_name'] = test_re['Date'].dt.day_name()


# %%
test_re.head()

,ID,timestamp,item,corporation,location,year,month,day,Date,week,day_name
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4,2023-03-04,9,Saturday
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5,2023-03-05,9,Sunday
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6,2023-03-06,10,Monday
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7,2023-03-07,10,Tuesday
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8,2023-03-08,10,Wednesday


In [17]:
# %%
from sklearn.preprocessing import LabelEncoder
train_x = train.drop(columns=['ID', 'timestamp', 'price(원/kg)','Date'])
train_y = train['price(원/kg)']
test_x = test_re.drop(columns=['ID', 'timestamp','Date'])
qual_col = ['item', 'corporation', 'location','day_name']

# train_x = pd.get_dummies(train_x, columns=qual_col, prefix=qual_col)
# test_x = pd.get_dummies(test_x, columns=qual_col, prefix=qual_col)

In [18]:
train_x = pd.concat([train_x, train_y], axis=1)

In [23]:
train_x.head()

,item,corporation,location,week,day_name,year,month,day,price(원/kg)
0,TG,A,J,1,Tuesday,2019,1,1,0.0
1,TG,A,J,1,Wednesday,2019,1,2,0.0
2,TG,A,J,1,Thursday,2019,1,3,1728.0
3,TG,A,J,1,Friday,2019,1,4,1408.0
4,TG,A,J,1,Saturday,2019,1,5,1250.0


In [20]:
test_x.head() #day_name 일치

,item,corporation,location,year,month,day,week,day_name
0,TG,A,J,2023,3,4,9,Saturday
1,TG,A,J,2023,3,5,9,Sunday
2,TG,A,J,2023,3,6,10,Monday
3,TG,A,J,2023,3,7,10,Tuesday
4,TG,A,J,2023,3,8,10,Wednesday


In [52]:
price.shape

(0, 9)

In [53]:
# import pycaret regression and init setup
from pycaret.regression import *

predictions = []

items = train_x['item'].unique()
corporations = train_x['corporation'].unique()
locations = train_x['location'].unique()

for item in items:
    for corporation in corporations:
        for location in locations:
            price = train_x[(train_x['item'] == item) &
                  (train_x['corporation'] == corporation) &
                  (train_x['location'] == location) &
                  (train_x['day_name']!='Sunday')]

            if len(price) <= 0: continue

            price = price[price['price(원/kg)']!=0]
                
            IQR = (price.describe()['price(원/kg)']['75%'] - price.describe()['price(원/kg)']['25%'])*1.5
            lower_bound = price.describe()['price(원/kg)']['25%'] - IQR
            upper_bound = price.describe()['price(원/kg)']['75%'] + IQR

            price['price(원/kg)'] = np.where((price['price(원/kg)'] > lower_bound) & (price['price(원/kg)'] < upper_bound), price['price(원/kg)'], 0)
            price['price(원/kg)'] = np.log(price['price(원/kg)'])
            
            try:            
                s = setup(price, target = 'price(원/kg)', 
                        session_id = 123, 
                        fold = 10, 
                        normalize = True, 
                        normalize_method = 'minmax',
                        train_size=0.9)
                
                # Best models 선택 (예: 상위 3개 모델 선택)
                reg = create_model('rf')
    
                price_test = test_x[(test_x['item'] == item) &
                      (test_x['corporation'] == corporation) &
                      (test_x['location'] == location)]
    
                prediction = list(np.exp(predict_model(reg, data = pd.DataFrame(price_test))['prediction_label'].values))
                predictions.extend(prediction)
                
            except:
                prediction = [0 for _ in range(price.shape[0])]
                predictions.extend(prediction)


,Description,Value
0,Session id,123
1,Target,price(원/kg)
2,Target type,Regression
3,Original data shape,"(1011, 9)"
4,Transformed data shape,"(1011, 14)"
5,Transformed train set shape,"(909, 14)"
6,Transformed test set shape,"(102, 14)"
7,Numeric features,4
8,Categorical features,4
9,Preprocess,True


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.2140,0.0847,0.2910,0.8419,0.0329,0.0273
1,0.1851,0.0611,0.2471,0.9033,0.0284,0.0240
2,0.1499,0.0468,0.2164,0.9254,0.0248,0.0193
3,0.1855,0.0626,0.2503,0.8964,0.0276,0.0230
4,0.1744,0.0548,0.2342,0.9020,0.0267,0.0224
5,0.2057,0.0923,0.3038,0.8427,0.0341,0.0259
6,0.2219,0.0826,0.2875,0.8615,0.0324,0.0282
7,0.2037,0.0894,0.2991,0.8485,0.0334,0.0259
8,0.1939,0.0696,0.2637,0.8873,0.0296,0.0246


,Description,Value
0,Session id,123
1,Target,price(원/kg)
2,Target type,Regression
3,Original data shape,"(1246, 9)"
4,Transformed data shape,"(1246, 14)"
5,Transformed train set shape,"(1121, 14)"
6,Transformed test set shape,"(125, 14)"
7,Numeric features,4
8,Categorical features,4
9,Preprocess,True


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,23:41:39
Status,. . . . . . . . . . . . . . . . . .,Finalizing Model
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Description,Value
0,Session id,123
1,Target,price(원/kg)
2,Target type,Regression
3,Original data shape,"(1174, 9)"
4,Transformed data shape,"(1174, 14)"
5,Transformed train set shape,"(1056, 14)"
6,Transformed test set shape,"(118, 14)"
7,Numeric features,4
8,Categorical features,4
9,Preprocess,True


...

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,23:42:24
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Description,Value
0,Session id,123
1,Target,price(원/kg)
2,Target type,Regression
3,Original data shape,"(710, 9)"
4,Transformed data shape,"(710, 14)"
5,Transformed train set shape,"(639, 14)"
6,Transformed test set shape,"(71, 14)"
7,Numeric features,4
8,Categorical features,4
9,Preprocess,True


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,23:42:25
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Description,Value
0,Session id,123
1,Target,price(원/kg)
2,Target type,Regression
3,Original data shape,"(585, 9)"
4,Transformed data shape,"(585, 14)"
5,Transformed train set shape,"(526, 14)"
6,Transformed test set shape,"(59, 14)"
7,Numeric features,4
8,Categorical features,4
9,Preprocess,True


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,23:42:26
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


In [39]:
tuned_reg = tune_model(reg, optimize = 'RMSE', fold=10, n_iter=20)
predictions = predict_model(tuned_reg, data = pd.DataFrame(test_x))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.0507,3.0731,1.7530,0.7960,0.5468,0.1601
1,1.1799,4.2667,2.0656,0.7254,0.5463,0.1823
2,1.1486,3.5424,1.8821,0.7745,0.5546,0.1659
3,1.1374,3.3327,1.8256,0.7822,0.6180,0.1344
4,1.2953,4.2701,2.0664,0.7247,0.6255,0.1657
5,1.1625,3.6440,1.9089,0.7619,0.5768,0.1789
6,1.6920,5.8095,2.4103,0.6194,0.7534,0.2417
7,1.4064,4.6488,2.1561,0.6808,0.6994,0.2173
8,1.0223,3.6189,1.9024,0.7582,0.5939,0.1322


Fitting 10 folds for each of 20 candidates, totalling 200 fits


In [56]:
predictions

[3918.207298383458,
 3552.7140096575827,
 3261.2420556319134,
 3141.493697812809,
 3079.6329703375486,
 3334.982654675934,
 3329.2554493549883,
 3305.5525578659763,
 3209.3079967245535,
 3330.2275617816085,
 3076.9401126999833,
 3010.767543583786,
 3287.0378238063527,
 6333.7994389797095,
 4749.0771098094965,
 4593.55148405973,
 4818.148025057625,
 4766.320150394544,
 4945.4416705632175,
 4702.737291428951,
 7385.74043382051,
 5662.569040087348,
 5080.772318505428,
 6658.114394274175,
 6209.121000541677,
 6220.616129769673,
 5786.207876211533,
 6179.804230420757,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0

In [50]:
test_re.shape

(1092, 11)

In [ ]:
result = pd.concat([test_re, predictions['prediction_label']], axis = 1)

result['answer'] = result['prediction_label'].copy()

del result['timestamp']
del result['item']
del result['corporation']
del result['location']
del result['year']
del result['month']
del result['day']
del result['Date']
del result['week']
del result['day_name']
del result['prediction_label']

In [ ]:
for i in range(len(result)):
    if (i-1) % 7 == 0: # 1, 8, 15, 22
        result.iloc[i,-1] = 0

result.head(20)

In [ ]:
result.to_csv('result_hard_tuened.csv', index = False)

In [ ]:
result

In [ ]:
train_x.to_csv('./processed_data/train.csv')
test_x.to_csv('./processed_data/test.csv')